In [1]:
import flash
import mlflow
import torch
from flash.core.data.utils import download_data
from flash.text import TextClassificationData, TextClassifier
import torchmetrics
from pytorch_lightning.core.memory import ModelSummary
import shutil
import tempfile
import os


In [2]:
os.environ["AWS_ACCESS_KEY_ID"] = "minio"
os.environ["AWS_SECRET_ACCESS_KEY"] = "minio123"
#os.environ["MLFLOW_S3_ENDPOINT_URL"] = "http://localhost:9000"
os.environ["MLFLOW_S3_ENDPOINT_URL"] = "http://ai-machine:9000"

In [3]:
def save_file_as_mlflow_artifact(file_content_string, file_name = 'model_summary.txt'):
    tempdir = tempfile.mkdtemp()
    try:
        summary_file = os.path.join(tempdir, file_name)
        with open(summary_file, "w") as f:
            f.write(file_content_string)

        mlflow.log_artifact(local_path=summary_file)
    finally:
        shutil.rmtree(tempdir)

In [4]:
download_data("https://pl-flash-data.s3.amazonaws.com/imdb.zip", "./data/")
datamodule = TextClassificationData.from_csv(
    input_fields="review",
    target_fields="sentiment",
    train_file="data/imdb/train.csv",
    val_file="data/imdb/valid.csv",
    test_file="data/imdb/test.csv"
)

C:\Users\Ila\AppData\Roaming\Python\Python38\site-packages\urllib3\connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'pl-flash-data.s3.amazonaws.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


./data/imdb.zip:   0%|          | 0/15575 [00:00<?, ?KB/s]

Using custom data configuration default-1d662a7890cd42e9


0 tables [00:00, ? tables/s]

Parameter 'function'=functools.partial(<function TextDataSource._transform_label at 0x000001B878EBD700>, {'negative': 0, 'positive': 1}, 'sentiment') of the transform datasets.arrow_dataset.Dataset._map_single couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is only showed once. Subsequent hashing failures won't be showed.


...

Using custom data configuration default-ee4a431508e7197f


0 tables [00:00, ? tables/s]

Dataset csv downloaded and prepared to C:\Users\Ila\.cache\huggingface\datasets\csv\default-ee4a431508e7197f\0.0.0\e138af468cb14e747fb46a19c787ffcfa5170c821476d20d5304287ce12bbc23. Subsequent calls will reuse this data.


  0%|          | 0/2500 [00:00<?, ?ex/s]

  0%|          | 0/3 [00:00<?, ?ba/s]

In [5]:
# if we want to track multiple metrics simultaneously, define a list of metrics and pass it to the classifier metric parameter
list_of_metrics = [torchmetrics.Accuracy(),
                    torchmetrics.F1(num_classes=datamodule.num_classes),
                    torchmetrics.Precision(num_classes=datamodule.num_classes),
                    torchmetrics.Recall(num_classes=datamodule.num_classes)]

In [6]:
# uncomment the following line and comment out the immediate next line to use a list of metrics instead of a single metric during model training
# classifier_model = TextClassifier(backbone="prajjwal1/bert-tiny", num_classes=datamodule.num_classes, metrics=list_of_metrics)

classifier_model = TextClassifier(backbone="prajjwal1/bert-tiny", num_classes=datamodule.num_classes, metrics=torchmetrics.F1(datamodule.num_classes))
trainer = flash.Trainer(max_epochs=3, gpus=torch.cuda.device_count())

Using 'prajjwal1/bert-tiny' provided by Hugging Face/transformers (https://github.com/huggingface/transformers).
Some weights of the model checkpoint at prajjwal1/bert-tiny were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.weight', 'cls.predictions.decoder.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification

In [7]:

EXPERIMENT_NAME = "dl_model_chapter03"
#mlflow.set_tracking_uri('http://localhost')
mlflow.set_tracking_uri('http://ai-machine')
mlflow.set_experiment(EXPERIMENT_NAME)
experiment = mlflow.get_experiment_by_name(EXPERIMENT_NAME)
print("experiment_id:", experiment.experiment_id)

experiment_id: 1


In [8]:
MODEL_ARTIFACT_PATH = 'dl_model'
REGISTERED_MODEL_NAME = 'nlp_dl_model'
with mlflow.start_run(experiment_id=experiment.experiment_id, run_name="chapter03") as dl_model_tracking_run:
    trainer.finetune(classifier_model, datamodule=datamodule, strategy="freeze")
    trainer.test()

    # mlflow log metrics
    cur_metrics = trainer.callback_metrics
    # Cast metric value as  float before passing into logger.
    metrics = dict(map(lambda x: (x[0], float(x[1])), cur_metrics.items()))
    mlflow.log_metrics(metrics)

    # mlflow log parameters
    params = {"epochs": trainer.max_epochs}
    if hasattr(trainer, "optimizers"):
        optimizer = trainer.optimizers[0]
        params["optimizer_name"] = optimizer.__class__.__name__
    if hasattr(optimizer, "defaults"):
        params.update(optimizer.defaults)
    # add hyper-parameters used in this training
    params.update(classifier_model.hparams)
    mlflow.log_params(params)

    # log model summary as an artifact
    summary = ModelSummary(classifier_model, max_depth=-1)
    save_file_as_mlflow_artifact(str(summary))

    # log and register the trained model
    mlflow.pytorch.log_model(pytorch_model=classifier_model, artifact_path=MODEL_ARTIFACT_PATH, registered_model_name=REGISTERED_MODEL_NAME)



  | Name          | Type                          | Params
----------------------------------------------------------------
0 | train_metrics | ModuleDict                    | 0     
1 | val_metrics   | ModuleDict                    | 0     
2 | model         | BertForSequenceClassification | 4.4 M 
----------------------------------------------------------------
258       Trainable params
4.4 M     Non-trainable params
4.4 M     Total params
17.545    Total estimated model params size (MB)


Validation sanity check: 0it [00:00, ?it/s]

Training: -1it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Testing: 0it [00:00, ?it/s]

--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'test_cross_entropy': 0.794507622718811, 'test_f1': 0.5368000268936157}
--------------------------------------------------------------------------------


Registered model 'nlp_dl_model' already exists. Creating a new version of this model...
2022/07/17 19:54:16 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.                     Model name: nlp_dl_model, version 4
Created version '4' of model 'nlp_dl_model'.


In [9]:
run_id = dl_model_tracking_run.info.run_id
print("run_id: {}; lifecycle_stage: {}".format(run_id,
    mlflow.get_run(run_id).info.lifecycle_stage))

run_id: d946a820957e451a847433a0ee4b445a; lifecycle_stage: active


In [10]:
logged_model = f'runs:/{run_id}/{MODEL_ARTIFACT_PATH}'
# Load model as a pytorch model, not as the pyfunc model
model = mlflow.pytorch.load_model(logged_model)
# To bypass a lightning-flash's bug, we need to set the stage to test so a loaded model can be used to do prediction
model.trainer.state.stage='test'
model.predict({'What a news!'})

['negative']